In [1]:
#!/usr/bin/env python3
import sys, os
import json
import numpy as np
import argparse
from parse import parse
import pandas as pd
import h5py
#!pip install ipympl==0.7.0 --user

import matplotlib.pyplot as plt
plt.style.use('../clint.mpl')

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
from pygama.io.daq_to_raw import daq_to_raw
from pygama.io.raw_to_dsp import raw_to_dsp

# Load Files

In [2]:
f_daq = '$CAGE_SW/nonlinearity/2021-7-28-CAGERun2'
out_dir = '$CAGE_SW/nonlinearity'
f_raw = 'test_run.lh5'

config = {'daq': 'ORCA'}
print(config)

{'daq': 'ORCA'}


In [3]:
# Create raw file if not already made
# daq_to_raw(f_daq, f_raw, subrun=1, out_dir='./', config=config)

In [4]:
f = h5py.File(f_raw)
wf = f['ORSIS3302DecoderForEnergy']['raw']['waveform']

# Make DataFrame

In [5]:
ndval = wf['values'][()]
df = pd.DataFrame(ndval)

# Plot Individual Waveforms

In [6]:
def plot_wf(df, ind, save=False, name=None):
    %matplotlib widget
    values = df.iloc[ind]
    samples = np.arange(len(values))
    plt.plot(samples, values)
    plt.xlabel('Sample')
    plt.ylabel('ADU')
    if save:
        name = name if name else 'adcnl_wf{ind}.png'
        plt.savefig(name)
    plt.show()

In [7]:
plot_wf(df, 3000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Histogram ADCs

In [8]:
values = np.ndarray.flatten(ndval)
counts = np.bincount(values)
bins = np.arange(-0.5, np.amax(values)+1.5)

In [26]:
%matplotlib widget
pgh.plot_hist(counts, bins)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Save histogram counts and bins
# np.save('adc_counts.npy', counts)
# np.save('adc_bins.npy', bins)

# Find Low Counts

In [11]:
low_wfs = np.unique(np.where(df < 5000)[0])
print(low_wfs)

[30650]


In [12]:
plot_wf(df, low_wfs[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Superpulses

In [13]:
#Remove wonky waveform
df_cor = df.drop(low_wfs[0])

In [14]:
def norm_df(df):
    subbed = df.sub(df.min(axis=1),axis=0)
    normed = subbed.div(subbed.max(axis=1), axis=0)
    return normed

def mean_df(df):
    return df.mean().to_frame().transpose()

In [16]:
superpulse = mean_df(norm_df(df))

In [17]:
superpulse

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.401718,0.417393,0.390143,0.397678,0.402225,0.417947,0.390667,0.398306,0.402578,0.418067,...,0.583106,0.591051,0.595515,0.610629,0.583818,0.591315,0.595578,0.611199,0.583911,0.397459


In [23]:
plot_wf(superpulse, 0, True, 'superpulse_all.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
df_low = df.nsmallest(int(len(df)/2), 0)
df_high = df.nlargest(len(df)-int(len(df)/2), 0)

In [53]:
df_low

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
192993,6097,6099,6095,6115,6130,6140,6152,6160,6169,6156,...,6210,6205,6201,6198,6187,6190,6193,6195,6197,6114
127872,6104,6104,6099,6107,6103,6128,6142,6140,6158,6165,...,6167,6188,6180,6189,6156,6153,6131,6135,6130,6126
202661,6104,6110,6104,6092,6108,6103,6115,6111,6127,6139,...,6185,6199,6200,6183,6169,6161,6186,6113,6213,6103
74749,6105,6080,6084,6094,6108,6117,6148,6155,6180,6179,...,6172,6161,6138,6126,6125,6128,6143,6144,6159,6118
347332,6105,6128,6129,6150,6147,6151,6154,6157,6160,6164,...,6099,6130,6138,6148,6150,6161,6163,6172,6168,6109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277029,32051,32046,32046,32032,32044,32041,32027,32033,32028,32022,...,32078,32088,32097,32083,32085,32076,32081,32056,32066,32045
284929,32051,32040,32035,32025,32027,32021,32037,32049,32065,32062,...,32082,32074,32076,32045,32075,32075,32075,32098,32103,32060
332463,32051,32063,32059,32064,32066,32082,32074,32096,32116,32129,...,32120,32127,32123,32139,32115,32108,32110,32115,32122,32063
4009,32052,32062,32068,32051,32069,32074,32063,32052,32059,32060,...,32124,32132,32148,32132,32144,32141,32142,32129,32117,32050


In [54]:
df_high

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
183307,58107,58101,58067,58052,58043,58042,58041,58033,58032,58035,...,58026,58038,58051,58067,58063,58058,58095,58074,58083,58128
223551,58102,58077,58063,58040,58022,57984,57984,57980,57983,57982,...,58046,58038,58059,58055,58058,58070,58079,58078,58065,58098
258793,58099,58102,58084,58086,58089,58070,58059,58049,58054,58014,...,58088,58074,58079,58057,58050,58039,58050,58050,58046,58098
301624,58099,58090,58077,58080,58075,58058,58052,58054,58059,58053,...,58071,58073,58103,58095,58103,58086,58095,58086,58089,58097
241182,58095,58100,58085,58089,58076,58065,58053,58055,58049,58058,...,58013,58011,58000,58056,58047,58054,58067,58103,58093,58086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88900,32053,32057,32050,32057,32044,32055,32033,32021,32012,32023,...,32109,32118,32130,32148,32126,32125,32110,32109,32092,32067
374846,32053,32070,32044,32044,32037,32045,32030,32026,32016,32026,...,32100,32093,32077,32068,32044,32027,32028,32032,32020,32053
384223,32053,32071,32058,32066,32054,32082,32045,32084,32068,32082,...,32038,32054,32071,32085,32084,32077,32075,32101,32094,32063
4009,32052,32062,32068,32051,32069,32074,32063,32052,32059,32060,...,32124,32132,32148,32132,32144,32141,32142,32129,32117,32050


In [20]:
superpulse_low = mean_df(norm_df(df_low))
superpulse_high = mean_df(norm_df(df_high))

In [24]:
plot_wf(superpulse_low, 0, True, 'superpulse_low.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
plot_wf(superpulse_high, 0, True, 'superpulse_high.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …